### 1. Data collection

### 1.1. Get the list of movies

In [24]:
# Importing all the libraries that are needed for solving this homework 
import urllib.request
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
import collections
import math
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
# Ist we decided to download data files on PC's of us separately
#than we come up with more efficient way of downloading the data 

In [26]:
# we know that we have 3 documents to parse. This function get the URL of the 3 documents provided, and store them in liste2
#we store the url in a dataframe to let us drop the url that does not work

In [ ]:
# In this part we work on the crawling of wikipedia we dowloaded the data from the website and stored it on local machine 

In [3]:
def file_to_parse(number_of_file_to_parse):    
    liste2 = []
    for k in range(1, number_of_file_to_parse+1):
        i = str(k)
        url = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies'+i+'.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for par in soup.select('a'):
            liste2.append(par.text)
    df= pd.DataFrame(liste2)
    df = df.drop(df.index[[9429, 9671,15520,15576,17725,18100,21267,23664,25240,25873,27675,27721,27768,28053,28180,28273,28378,29229]])
    df = df.reset_index()
    return(df)

### 1.3 Parse downloaded pages

In [27]:
# We need this function to separate some proper names that will appear in some values of the dictionary,
# for example the value associated to the key "Starring"
def SepName(s):

    for i in range(1, len(s)):
        if s[i].isupper() and s[i - 1] != " ":
            s = s[:i] + " " + s[i:]

    return s

In [ ]:
# this part trys to extract the intro and plot of the film, if they aren't into the page they are set to NA

In [7]:
# This function parses the intro
def Intro( html, dic):

    # We start from the table because the intro is always on the same level of the table
    start = html.find( "table", {"class" : "infobox vevent"} )

    tag = "p"
    intro = ""

    # Then we add all the intro paragraphs text in the string "intro";
    # when the first tag different from "p" is found, the loop ends.
    # There is always an intro so we don't need to set the intro to NA in advance
    while tag == "p":

        paragraph = start.find_next_sibling()
        tag = paragraph.name

        if tag == "p":
            intro += paragraph.text

        start = paragraph

    dic["Intro"] = intro


In [8]:
# This function parses the plot
def Plot( html, dic):

    dic["Plot"] = "NA"

    # Here we start from "span" {"class": "mw-headline"} because it indicates the title of a new Wikipedia section
    start = html.find("span", {"class": "mw-headline"})

    # If there is a section linked to the plot, we take it; otherwise we leave the value set to NA and we do nothing
    # in the exception with "pass"
    try:

        # We scroll through all the sections until we find one named in the following way.
        # These are all the possible titles of the sections linked to the plot, found by examining all the files
        while start["id"] != "Plot" and start["id"] != "Plot_summary" and start["id"] != "Premise" and start[
            "id"] != "Summary":
            start = start.find_next("span", {"class": "mw-headline"})

        # We go back up the hierarchy of a level because the paragraphs containing the plot are on the same level
        # as the "h2" tags, which contain "span": this because later we want to scroll the siblings
        # in the same way proposed in Plot function
        start = start.find_parent()
        tag = "p"
        plot = ""

        while tag == "p":

            paragraph = start.find_next_sibling()
            tag = paragraph.name

            if tag == "p":
                plot += paragraph.text

            start = paragraph

        dic["Plot"] = plot

    except:
        pass


In [28]:
# extracting data from crawled HTML pages and transform it into a film.tsv file with the extracted informations as specified
# into the requests

In [10]:
# This function parses the info box
def Info( html, dic):

    # Here we start from the info box itself
    info_box = html.find("table", {"class": "infobox vevent"})

    # The first content with the "tr" tag is always the name of the movie
    start = info_box.find("tr")
    dic["Film name"] = start.text

    # We find the next "th" tags because they are the ones containing the requested information:
    # usually there is an image of the movie poster that in this way is skipped since it has tag "td"
    start = start.find_next("th")
    start = start.find_parent()
    tag = "tr"

    # We set all values ​​to NA in advance
    dic["Director"] = dic["Producer"] = dic["Writer"] = dic["Starring"] = dic["Music"] = dic["Release date"] = dic[
        "Runtime"] = dic["Country"] = dic["Language"] = dic["Budget"] = "NA"

    # Here too we scroll all the "tr" tags and analyze their content
    while (tag == "tr"):

        line = start.find_next_sibling()

        # Instead the information is always contained at a lower level than the variable "line".
        # If these informations are somehow not contained in the html file,
        # we risk an infinite loop: the exception is for this
        try:
            info = line.find_next()
        except:
            break

        tag = line.name
        start = line

        # We save the information of interest in the dictionary
        if info.text == "Directed by":
            dic["Director"] = SepName(info.find_next("td").text)

        elif info.text == "Produced by":
            dic["Producer"] = SepName(info.find_next("td").text)

        elif info.text == "Written by":
            dic["Writer"] = SepName(info.find_next("td").text)

        elif info.text == "Starring":
            dic["Starring"] = SepName(info.find_next("td").text)

        elif info.text == "Music by":
            dic["Music"] = SepName(info.find_next("td").text)

        elif info.text == "Release date":
            dic["Release date"] = info.find_next("td").text

        elif info.text == "Running time":
            dic["Runtime"] = info.find_next("td").text

        elif info.text == "Country":
            dic["Country"] = info.find_next("td").text

        elif info.text == "Language":
            dic["Language"] = SepName(info.find_next("td").text)

        elif info.text == "Budget":
            dic["Budget"] = info.find_next("td").text

### 2. Search Engine

In [30]:
#this part clean the data for search engine we remove all the data that was not helpful in search engine while 
#searching through documents e.g.stopwords, punctuation and also did Stemming

In [13]:
def preprocess(k):
    
    
    k = k.lower() # make all the word turn into lower case
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(k)
    stop_words = set(stopwords.words('english'))
    text = result
    new_sentence =[]
    for w in text:
        if w not in stop_words: 
            new_sentence.append(w) #drop the stopwords of our string
    ps = PorterStemmer()
    final = []
    for word in new_sentence:
        final.append(ps.stem(word)) #get the root form of each word
    return(final)


### 2.1. Conjunctive query



In [29]:
# This part makes the indexing of the extracted informations 

### 2.1.1) Create your index!

In [16]:
# here we create the index 
def index_1():

    listemot = [] #will contain the words 
    listedoc = [] # will be a list of lists. Each list in this list is link to a unique word, and will contain the document in which the word appears

    for k in range(0,29982):
        if k%100 == 0 : 
            print(k) # indicator to see at which step we are during the computing
        try : 
            nbr = str(k)
            with open(r'C:\Users\danyl\OneDrive\Documents\tsv\document_'+nbr+'.tsv', encoding = 'utf8') as tsvfile:
                reader = csv.reader(tsvfile, delimiter='\t')
                for row in reader:
                    if len(row)>0:
                        a = row[1] # we get the intro
                        b = row[2] #we get the plot
            b = preprocess(b) # we drop the stopwords, ponctuation, and turn words into root form
            a = preprocess(a)
            for element in b: #we iterering over the words in the plot
                if element in listemot: #we check if the word is already in the list
                    i = listemot.index(element) #if it is, we get its index 
                    if nbr not in listedoc[i]: #we check if we don't already refer that the word is in this document
                        listedoc[i].append(nbr) #we save the tsv number in a list which is link to this specific word.
                else : 
                    listemot.append(element) #we add the word in the list if it is not already in
                    ldoc = [nbr] #we store the number of the tsv file
                    listedoc.append(ldoc) # we store the new list in the list of document
            for element in a: #we do the same thing we the intro, with the same logic
                if element in listemot:
                    i = listemot.index(element)
                    if nbr not in listedoc[i]:
                        listedoc[i].append(nbr)
                else : 
                    listemot.append(element)
                    ldoc = [nbr]
                    listedoc.append(ldoc)
        except:
            pass
    
    a = {}
    # we create a dictionnary. the keys of the dictionnary is the index of each word, and the value is a list of document in which the word appears
    for k in range(0,len(listemot)):
        a[k] = listedoc[k]
    
    return(a,listemot, listedoc)
            

In [18]:
# this code is similar to the code we have done to build the first index. We just have two difference
def index_2():
    listemot2 = []
    listedoc2 = []
    dic_words_doc = {} # will contain, for each document, the number of words it contain

    for k in range(0,29982):
        if k%100 == 0:
            print(k)
        try : 
            nbr = str(k)
            with open(r'C:\Users\danyl\OneDrive\Documents\tsv\document_'+nbr+'.tsv', encoding = 'utf8') as tsvfile:
                reader = csv.reader(tsvfile, delimiter='\t')
                for row in reader:
                    if len(row)>0:
                        a = row[1]
                        b = row[2]
            b = preprocess(b)
            a = preprocess(a)
            for element in b: 
                if element in listemot2:
                    i = listemot2.index(element)
                    listedoc2[i].append(nbr) #the difference is that even if we have already stored the document that contain the word, we are going to store it again if we have the same word several time in the same document. So we have informations on the occurence of each word in each document
                else : 
                    listemot2.append(element)
                    ldoc = [nbr]
                    listedoc2.append(ldoc)
            for element in a: 
                if element in listemot2:
                    i = listemot2.index(element)
                    listedoc2[i].append(nbr)
                else : 
                    listemot2.append(element)
                    ldoc = [nbr]
                    listedoc2.append(ldoc)
            dic_words_doc[k] = len(a) + len(b) # here we store the number of words in the intro+plot
        
        except : 
            pass
    listeinverted = []

    # here we want to create a dictionnary in which the key will be the index of the word, and the value will be a dictionnary (key = document, value = tf-idf)

    for k in range(len(listedoc2)): #listedoc2 contain lists of documents link a unique word
        i = collections.Counter(listedoc2[k]) # We now have a dictionnary (key = document, value = occurence of the word in this document)
        dic = {} 
        for key, value in i.items(): # we go into the dictionnary i we created
            number = dic_words_doc[int(key)] # we get the number of words of the document which is the key of the dictionnary
            dic[key] = (value/number)*(1+math.log(float(29981/(len(i))))) # we calculate the tf-idf of each document
        listeinverted.append(dic) #we store the dic that refers to a word and that containt (key = document, value = tf_idf) in a list
    

    # here we create the dictionnary (key = index_word, value = dictionnary(key = document, value = tf-idf))
    dicInverted = {}

    for k in range(0,len(listemot2)):
        dicInverted[k] = listeinverted[k]
    return(dicInverted, listemot2, listedoc2, dic_words_doc)
    

### 4. Algorithmic question

In [31]:
# defined a function to find maximum of two values 

In [20]:
def maximum(x,y):
    if x>y:
        return x
    else:
        return y

In [32]:
def long_palindromic_seq(str): 
    n = len(str) 
  
    # Creating a table to store results of subproblems 
    L = [[0 for x in range(n)] for x in range(n)] 
  
    # Strings of length 1 are palindrome of length 1 
    for i in range(n): 
        L[i][i] = 1
   
    for c in range(2, n+1): 
        for i in range(n-c+1): 
            j = i+c-1
            if str[i] == str[j] and c == 2: 
                L[i][j] = 2
            elif str[i] == str[j]: 
                L[i][j] = L[i+1][j-1] + 2
            else: 
                L[i][j] = maximum(L[i][j-1], L[i+1][j]); 
  
    return L[0][n-1] 



In [33]:

# testing the code

In [23]:
seq = "DATAMININGSAPIENZA"
print("The length of the Longest palindromic sequence is " + str(long_palindromic_seq(seq)))

The length of the Longest palindromic sequence is 7
